# MNIST Classifier Model with Multiple Logistic Regression

## Import Libraries

In [127]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

## Define Functions

In [128]:
# One-hot Encoding for lables
def one_hot_encoding(labels, class_num):
    data_num = len(labels) # 60000, 10000
    result = np.zeros((data_num, 10))
    
    for idx in range(data_num):
        result[idx][labels[idx]] = 1
        
    return result

def Z(X, W, B):
    return tf.add(tf.matmul(X, W), B)

def softmax(Z):
    return tf.nn.softmax(Z)

def cross_entropy(Y, softmax):
    return -tf.reduce_sum(Y * tf.log(softmax), axis=1)

## Initialize Variables

In [129]:
# Load MNIST dataset from implementing Keras API
(x_train, y_train), (x_test, y_test) = mnist.load_data() # 60000 training set, 10000 testing set

print(f'x_train: {x_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'x_test: {x_test.shape}')
print(f'y_test: {y_test.shape}')

x_train: (60000, 28, 28)
y_train: (60000,)
x_test: (10000, 28, 28)
y_test: (10000,)


## Preprocess Data

In [142]:
# Reshape train data and test data / Divide by 255 to normalize (the largest value of RGB is 255).
x_train = x_train.reshape((60000, 28*28)) / 255 # (60000, 784)
x_test = x_test.reshape((10000, 28*28)) / 255 # (10000, 784)

# Hyperparameters
class_num = 10
learning_rate = 0.001
batch_size = 100
batch_number = int(len(x_train) / batch_size)
epochs = 50

In [131]:
# Make Y values using One-Hot encoding
y_train = one_hot_encoding(y_train, class_num)
print(y_train[:4])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Build Model

In [143]:
# Initialize tensors with proper size and data type (float32)
X = tf.placeholder(tf.float32, [None, 28 * 28])
Y = tf.placeholder(tf.float32, [None, class_num])
W = tf.Variable(np.zeros((784, 10), np.float32))
B = tf.Variable(np.zeros((len(x_train), 10),np.float32))

In [133]:
softmax = softmax(Z(X, W, B))
cost = cross_entropy(Y, softmax)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [144]:
with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for i in range(batch_number):
            offset = i * epoch
            x = x_train[offset: offset + batch_size]
            y = y_train[offset: offset + batch_size]
            sesh.run(optimizer, feed_dict={X: x, Y:y})
            c = sesh.run(cost, feed_dict={X:x, Y:y})
        
        if not epoch % 2:
            print(f'epoch:{epoch:2d} cost={c:.4f}')
            
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    acc = accuracy.eval({X: x_test, Y: y_test})
    print(f'Accuracy: {acc * 100:.2f}%')

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_16' with dtype float and shape [60000,784]
	 [[node Placeholder_16 (defined at C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\site-packages\tensorflow_core\python\framework\ops.py:1762) ]]

Original stack trace for 'Placeholder_16':
  File "C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\KyochulJang\anaconda3\envs\dlstudy\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C: